<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Decisions with Semantic Functions 

Decision Intelligence applied in this module:  
* Listing of various decision-making frameworks with their descriptions 
* Listing types of decision-making frameworks dynamically based on the type  

Semantic Kernel Functions are the core building blocks of functionality for intelligent AI orchestration. Semantic Kernel functions usually have a single responsibility to perform. For example, a Sementic Kernel function can: send an e-mail, call an API, recommend a reasoning framework for a high-stakes decision etc. 

What makes Semantic Kernel functions so special? Can't one just produce similar outcomes by using a prompt in an LLM or just writing a software program? In order to answer that, one has to look at what makes GenAI so innovative. The GenAI models have a unique ability to process instructions with reasoning and logic. This allows these models to behave almost like a human decision maker. Even with basic prompts, GenAI models perform reasonably well. However, providing GenAI models with additional functions allows them to gain access to business processes, data and basically anything a GenAI model should consider when performing AI orchestration. 

For example, imagine you want to prepare a great Thanksgiving dinner and want to get help from a GenAI cooking application to create a new recipe. You enter what you want todo and it comes up with the most delicious looking feast for Thanksgiving. However, there is a problem it used ingredients and recommended using cooking appliances that you do not own! You could enter all of the ingredients and keep prompting until it narrowed down the what you could realistically make. Wouldn't it be better if the GenAI cooking application had access to: your available ingredients, your time availability, kitchen appliances and even your allergic preferences. Now the GenAI model can craft a Thanksgiving feast understanding the parameters and data without having to be guided. This is exactly what Semantic Kernel functions provide.

Semantic Kernel functions come in two flavors: 
* Semantic functions
* Native functions

Semantic functions are basically **prompt instructions** that help guide GenAI LLMs around the specific guidelines it should adhere to when building the AI orchestration. Therefore, semantic functions are likely the first place most AI architects/engineers start when composing AI orchestration with Semantic Kernel. Semantic functions can also take in parameters for dynamic creaation of instructions.  

Below is an example of semantic function that can be used for facilitating decision-making. Note the **{{$decisionToMake}}** input where the parameter can be dynamically passed in to provide additional specificity.

```javascript
[DECISION FRAMEWORKS TO USE]
Price's Law
Pareto Principle
Laplace Rule of Succession
Eisenhower Matrix
Median Rule of Five
Second Order Thinking

[DECISION GUIDANCE]
Try to use the best fitting framework.  
Prefer using quantitative decision frameworks rather qualitative ones.  
Use the code interpreter to validate quantitative reasoning.

Use the decision frameworks above to help the user make the following decision:
{{$decisionToMake}}
```  

The image below illustrates the core value of semantic functions. 

<img style="display: block; margin: auto;" width ="500px" src="https://learn.microsoft.com/en-us/semantic-kernel/media/semantic-function-explainer.png">

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [1]:
#r "nuget: Microsoft.Extensions.Configuration, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.45"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.45-alpha"

using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using System.IO;

var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.0 Microsoft.Extensions.Configuration.Json, 9.0.0 Microsoft.SemanticKernel, 1.45.0 Microsoft.SemanticKernel.Plugins.Core, 1.45.0-alpha

In [2]:
Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

Using Azure OpenAI Service


### Step 2 - Create a Semantic Kernel - Semantic Function 

> 📜 "Decision frameworks are like maps. Use them to navigate complex decision-making terrain.  
>
> -- <cite>Unknown</cite> 

Semantic functions are constructs inside Semantic Kernel that are basically just prompt instructions. What makes the special is that they can be exposed as functions, thus it makes re-usability easy. Furthermore, future modules will show how they can faciliate composability into plugins. 

In [3]:
// Simple prompt to list some decision frameworks this GenAI LLM is familiar with
var decisionPromptTemplate = """
Identify and list various decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each framework supports better analysis and reasoning in different decision-making scenarios.
""";

// Takes a regular prompt and creats a function that can be used to invoke the model
var semanticDecisionFunction = semanticKernel.CreateFunctionFromPrompt(decisionPromptTemplate);

await foreach (var streamChunk in semanticKernel.InvokeStreamingAsync(semanticDecisionFunction))
{
   Console.Write(streamChunk);
}

Here’s a list of various decision-making frameworks along with brief descriptions of how they enhance analysis and reasoning in different scenarios:

---

### **1. Decision Tree Analysis**
- **Description**: A visual framework that maps out possible decisions and their potential outcomes, including probabilities and payoffs.
- **How It Supports Better Decisions**: Helps to break down complex decisions into manageable components, allowing for structured evaluation of options and outcomes, especially under uncertainty.

---

### **2. SWOT Analysis (Strengths, Weaknesses, Opportunities, Threats)**
- **Description**: A structured approach to evaluating internal capabilities (strengths, weaknesses) and external factors (opportunities, threats).
- **How It Supports Better Decisions**: Encourages holistic assessment of a situation to ensure all relevant internal and external factors are considered, especially for strategic planning.

---

### **3. Cost-Benefit Analysis**
- **Description**: A 

### Step 3 - Semantic Function Dynamic Decision Intelligence  

> 📜 "If the only tool you have is a hammer, you tend to see every problem as a nail."
>
> -- <cite>Abraham Maslow (Renowned American psychologist)</cite> 

Semantic functions can be dynamically composed using parameter placeholders, which can then be dynamically rendered using Kernel arguments. This allows the ease of passing in parameters and execution settings into the semantic functions.

Execute the cell below to view how the prompt can dynamically instruct the LLM to retrieve different types of decision-making frameworks. 

In [5]:
// Takes a prompt with input variables and creates a function that can be used to invoke the GenAI model
var decisionPromptTemplate = """
Identify and list {{$numberOfFrameworks}} decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each framework supports better analysis and reasoning in {{$decisionType}} decision-making scenarios.
""";

// Create the OpenAI prompt execution settings, try changing the settings to see how the output changes
// Try different settings (Temperature, FrequencyPenalty etc) to see how they affect the quality of the generated text
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 750, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

var semanticDecisionFunctionWithInput = semanticKernel.CreateFunctionFromPrompt(decisionPromptTemplate, openAIPromptExecutionSettings);

// Dynamically set the number of frameworks and decision type -> strategic
var kernelArguments = new KernelArguments();
kernelArguments.Add("numberOfFrameworks", 3);
kernelArguments.Add("decisionType", "Long-Term strategic with probabilistic outcomes");

// Stream the output of the model
await foreach (var streamChunk in semanticKernel.InvokeStreamingAsync(semanticDecisionFunctionWithInput, kernelArguments))
{
   Console.Write(streamChunk);
}

Here are three decision-making frameworks that can enhance the quality of decisions, particularly in long-term strategic scenarios with probabilistic outcomes:

---

### 1. **Expected Utility Theory (EUT)**  
**Description:**  
Expected Utility Theory is a mathematical framework used to make decisions under uncertainty by evaluating the potential outcomes of different choices based on their probabilities and the decision-maker's preferences (or utility).  

**How it supports better analysis and reasoning:**  
- **Quantifies trade-offs:** EUT helps decision-makers weigh the benefits and risks of various options by assigning numerical values to outcomes and their probabilities.  
- **Incorporates uncertainty:** By explicitly considering probabilistic outcomes, it ensures that decisions are not based solely on the most likely scenario but also account for less likely but impactful events.  
- **Aligns with long-term goals:** The utility function can be tailored to reflect long-term strate

In the example below, the number of frameworks to return has been changed as well as the type has been changed to **Quick to Implement for rapid Dicision-Making"**.  

In [6]:
// Dynamically set the number of frameworks and decision type -> requring fast action
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings);
kernelArguments.Add("numberOfFrameworks", 5);
kernelArguments.Add("decisionType", "Quick to Implement for rapid Dicision-Making");

// Stream the output of the model
await foreach (var streamChunk in semanticKernel.InvokeStreamingAsync(semanticDecisionFunctionWithInput, kernelArguments))
{
   Console.Write(streamChunk);
}

Here are five decision-making frameworks that can enhance the quality of decisions, particularly in rapid decision-making scenarios:

---

### 1. **OODA Loop (Observe, Orient, Decide, Act)**  
**Description:**  
The OODA Loop is a cyclical process designed for rapid decision-making, originally developed for military strategy. It involves four steps:  
- **Observe:** Gather relevant information about the situation.  
- **Orient:** Analyze the information and understand the context.  
- **Decide:** Choose the best course of action based on the analysis.  
- **Act:** Implement the decision quickly.  

**How it supports better analysis and reasoning:**  
The OODA Loop emphasizes agility and adaptability, enabling decision-makers to respond quickly to changing circumstances. It ensures that decisions are informed by real-time data and situational awareness, reducing the risk of delays.

---

### 2. **Eisenhower Matrix (Urgent-Important Matrix)**  
**Description:**  
This framework helps pri

#### Recommendation Decision Scenario with Dynamic Decision Intelligence

In the below code a decsion-analysis scenario is introduced that uses dynamic inputs to personalize the decision recommendation dynamically. The more specific information that provides contextual information to the Generative AI model can great improve the specific recommendations. 

**Decision Scenario:** Michael is a 35-year-old professional chef who is considering opening his own restaurant. This is a significant life decision that could greatly impact his career and personal life. Michael is looking for a recommendation for an approach (decision) for this potentially life-changing decision. 

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Scenarios/Scenario-RestaurantDecision.png">

Three factors will be considered for this scenario that the user will be prompted for: 
1) Michael's Total Net Worth in dollars (enter number)
2) Level of competition with other restaurants (low, medium, high) 
3) Level of support from Michael's friends and family (low, medium, high) 

**Various Inputs:** Vary the input of net worth, restaurant competition and level of family support to note the various outcomes.  

In [7]:
// Import the Microsoft.DotNet.Interactive namespace for user input
using Microsoft.DotNet.Interactive;

var totalNetWorth = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("Micheal's total net worth in dollars: ");
var levelOfCompetition = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("Level of competition with other restaurants (Low, Medium, High): ");
var levelOfFamilySupport = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("Level of support from Michael's friends and family (Low, Medium, High): ");

Console.WriteLine($"Michael's Net Worth: {totalNetWorth}");
Console.WriteLine($"Level of Restaurant Competition: {levelOfCompetition}");
Console.WriteLine($"Michael's level of Family Support: {levelOfFamilySupport}");
Console.WriteLine();

// Takes a prompt with input variables and creates a function that can be used to invoke the model
var restaurantDecisionRecommendation = """
Michael is a 35-year-old professional chef who is considering opening his own restaurant. 
This is a significant life decision that could greatly impact his career and personal life. 
Michael is looking for a recommendation on how to approach this.
Some key information about Michael:
- Michael's total net worth is (in US dollars) ${{$totalNetWorth}}.
- The level of competition with other restaurants is {{$levelOfCompetition}}.
- The level of support from Michael's friends and family is {{$levelOfFamilySupport}}.

Based on this information, what recommendation would you give to Michael regarding opening his own restaurant?
""";

// Create the OpenAI prompt execution settings, try changing the settings to see how the output changes
// Try different settings (Temperature, FrequencyPenalty etc) to see how they affect the quality of the generated text
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 750, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

var semanticRestaurantDecisionRecommendation = semanticKernel.CreateFunctionFromPrompt(restaurantDecisionRecommendation, openAIPromptExecutionSettings);

// Dynamically set the number of frameworks and decision type -> strategic
var kernelArgumentsRestaurantDecisionRecommendation = new KernelArguments();
kernelArgumentsRestaurantDecisionRecommendation.Add("totalNetWorth", totalNetWorth);
kernelArgumentsRestaurantDecisionRecommendation.Add("levelOfCompetition", levelOfCompetition);
kernelArgumentsRestaurantDecisionRecommendation.Add("levelOfFamilySupport", levelOfFamilySupport);

// Stream the output of the model
await foreach (var streamChunk in semanticKernel.InvokeStreamingAsync(semanticRestaurantDecisionRecommendation, kernelArgumentsRestaurantDecisionRecommendation))
{
   Console.Write(streamChunk);
}


Michael's Net Worth: 500000
Level of Restaurant Competition: high
Michael's level of Family Support: high

Opening a restaurant is a significant and complex decision that requires careful consideration of financial, personal, and market factors. Based on the information provided, here is a structured recommendation for Michael:

### 1. **Evaluate Financial Readiness**
   - **Net Worth:** With a net worth of $500,000, Michael has a solid financial foundation. However, opening a restaurant can be expensive, with costs for leasing, equipment, staffing, inventory, marketing, and more. He should:
     - Determine how much of his net worth he is willing to invest without jeopardizing his financial stability.
     - Consider securing additional funding (e.g., loans, investors) to reduce personal financial risk.
     - Create a detailed budget and financial plan, including a contingency fund for unexpected expenses.

### 2. **Conduct Market Research**
   - **Competition:** The high level of co